In [3]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands solution
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def calculate_angle(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    cos_theta = dot_product / (norm_v1 * norm_v2)
    cos_theta = np.clip(cos_theta, -1.0, 1.0)  # Ensure value is within valid range for arccos
    angle = np.arccos(cos_theta)
    return np.degrees(angle)

def track_wrist_rotation(hand_landmarks, initial_position, cumulative_angle):
    current_position = np.array([hand_landmarks.x, hand_landmarks.y])
    vector_initial = np.array([1, 0])  # Reference vector along the x-axis
    vector_current = current_position - initial_position

    angle = calculate_angle(vector_initial, vector_current)

    cross_product = np.cross(vector_initial, vector_current)
    if cross_product > 0:
        cumulative_angle += angle
    else:
        cumulative_angle -= angle

    return cumulative_angle % 360

cap = cv2.VideoCapture(0)
left_wrist_count = 0
right_wrist_count = 0
left_wrist_done = False
right_wrist_done = False
left_wrist_initial = None
right_wrist_initial = None
left_cumulative_angle = 0
right_cumulative_angle = 0

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]

                if wrist.x < 0.5:  # Right wrist
                    if right_wrist_initial is None:
                        right_wrist_initial = np.array([wrist.x, wrist.y])
                    else:
                        right_cumulative_angle = track_wrist_rotation(wrist, right_wrist_initial, right_cumulative_angle)
                        if right_cumulative_angle >= 360:
                            right_wrist_count += 1
                            right_cumulative_angle = 0  # Reset after a full rotation

                    if right_wrist_count >= 5:
                        right_wrist_done = True
                        cv2.putText(image, 'Good job! Right wrist movement complete.', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                if wrist.x > 0.5:  # Left wrist
                    if left_wrist_initial is None:
                        left_wrist_initial = np.array([wrist.x, wrist.y])
                    else:
                        left_cumulative_angle = track_wrist_rotation(wrist, left_wrist_initial, left_cumulative_angle)
                        if left_cumulative_angle >= 360:
                            left_wrist_count += 1
                            left_cumulative_angle = 0  # Reset after a full rotation

                    if left_wrist_count >= 5:
                        left_wrist_done = True
                        cv2.putText(image, 'Good job! Left wrist movement complete.', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        if left_wrist_done and right_wrist_done:
            cv2.putText(image, 'Both wrists exercise complete!', (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.imshow('Wrist Movement', image)
            cv2.waitKey(2000)
            break

        cv2.imshow('Wrist Movement', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


c:\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 